In [1]:
from IPython.display import display, HTML
display(HTML('''
<style>
.jp-Cell-outputWrapper .jp-Placeholder {
    display: none;
}
</style>
'''))

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2023</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Типы - Введение</b><br/>
    <br/>
    <font>Никита Бондарцев</font><br/>
</center>

### static typing vs dynamic typing

<center><img src="images/pills.jpg" width=700px/></center>

### Dynamic typing

Проверяется тип аргумента **в Runtime**. Тип ассоциирован с объектом


In [1]:
### Dynamic typing
def f(a: int) -> None:
    # print(a + 10)  # Throws TypeError
    a.some_func()  # Throws AttributeError

def g() -> None:
    print("Runtime")
    f("hello")

g()

Runtime


AttributeError: 'str' object has no attribute 'some_func'

### Static typing


Проверяется тип аргумента **до Runtime**. Тип ассоциирован с переменной




In [4]:
%%typecheck

### Static typing
def f(a: int, b, c) -> None:
    print(a + 10)
    a.some_func()   # Mypy error

def g() -> None:
    print("Runtime")
    f("hello")      # Mypy error

UsageError: Cell magic `%%typecheck` not found.


### Плюсы static/dynamic типизации

Static typing:
* **Видим проблему в месте вызова**, а не где-то в недрах
* Базово **документирует функцию** улучшая читабельность
* Помогает **структурировать код** (понять что на входе страшный динозавр и стоит зарефакторить)
* (не питон) Уменьшает нагрузку на runtime, так как **типы заранее проверяются** и можно эффективнее хранить значения

Dynamic typing:
* Можно не тратить **время на аннотацию типов**
* Библиотеки могут **оперировать любым множеством входов** и принимать решение про тип в рантайме

It should also be emphasized that **Python will remain a dynamically typed language, and the authors have no desire to ever make type hints mandatory, even by convention.**

Guido van Rossum, [PEP 484](https://peps.python.org/pep-0484/#non-goals)

<center><img src="images/static_code_analysis.jpg" width=700px/></center>

### Отлично, убедили, хочу тайпхинты. Но как?

<center><img src="images/awakening.jpg" width=700px/></center>

#### Типизации подлежат:

- аргументы функции/метода
- возвращаемые значения функции/метода
- переменные/атрибуты*

\* но достаточно часто типы локальных переменных могут быть выведены из кода

#### В качестве тайпхинтов можно использовать

* встроенные классы (list[str], dict[str,int], int, float, etc)
* пользовательские классы (class A, class B)
* абстрактные базовые классы (collections.abc.Mapping, collections.abc.Callable)
* типы из types/typing (types.FunctionType, typing.IO)
* None

### А можно не все-все типизировать? Gradual typing

Концепция, которая позволяет части программы быть динамически типизирована, а части - статически. Но есть нюансы

<center><img src="images/merged_pill.jpg" width=700px/></center>

### Как программно узнать типы функции?

(непонятно зачем это может понадобиться, но вот)

#### Читерим

In [ ]:
def f(a: int) -> None:
    pass

f.__annotations__  # в какой-то из очередных релизов может сломаться

#### Как на самом деле надо

In [ ]:
import typing

def f(a: int) -> None:
    pass

typing.get_type_hints(f)

### Хорошо, тайпинги функций я сам задавал, я их знаю, а что переменные?

In [ ]:
lst: list[int] = [1, 2, 3]
lst.__annotations__

In [ ]:
import typing
typing.get_type_hints(lst)

In [ ]:
%%typecheck
lst: list[int] = [1, 2, 3]
reveal_type(lst)

In [ ]:
# но в рантайме так не сработает
reveal_type(lst)

In [ ]:
typing.reveal_type(lst)  # с питона 3.11

In [ ]:
type(lst)

### Как работает проверка типов?

- типы проверяются не по реальному рантайм типу объекта, а по значению тайпхинта
- когда происходит присвоение переменной/биндинг аргумента функции, mypy проверяет, что передан подтип типа переменной/параметра
- если тип переменной не указан, тип переменной выводится по самому узкому возможному типу
- когда объявляется класс-наследник, mypy проверяет что наследник, действительно, является подтипом родителя


Определение подтипа:
* ∀ A: A -> A
* A -> B  =>  A.values ⊇ B.values
* A -> B  =>  A.functions ⊆ B.functions

\* логика такова, что в переменную a типа А можно положить объект любого его подтипа, и код, предполагающий что `a` имеет тип `A`, не сломается


### Но как же Gradual typing?

Если тип чего-либо не указан, то этому объекту автоматически присваивается специальный тип Any. Any имеет свойства, которые можно интерпретировать как то, что Any является подтипом любого типа

* A -> B => A ~> B
* ∀ A: Any ~> A
* ∀ A: A ~> Any  


### Насколько плохо не указывать часть типов, ч1

In [ ]:
%%typecheck
# Задаем все типы
def f(a: int) -> None:
    reveal_type(a)
    print(a << 10)

def g() -> None:
    b = 1.0             
    reveal_type(b)
    f(b)

In [ ]:
%%typecheck
# Не задаем типы в f
def f(a):
    c: int = a
    reveal_type(a)
    reveal_type(c)
    print(a << 10)

def g() -> None:
    b = 1.0
    reveal_type(b)
    f(b)

### Насколько плохо не указывать часть типов, ч2

In [ ]:
%%typecheck
# Не задаем типы в g
def f(a: int) -> None:
    reveal_type(a)
    print(a << 10)

def g():
    b = 1.0
    reveal_type(b)
    f(b)

In [ ]:
%%typecheck
# Задаем типы для f частично (без a)
def f(a) -> None:
    reveal_type(a)
    print(a << 10)

def g() -> None:
    b = 1.0
    reveal_type(b)
    f(b) 

### Вывод типов при присваивании, ч1

In [ ]:
%%typecheck
# Просто присваивание
class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass

b = B()
reveal_type(b)

b.bm()

### Вывод типов при присваивании, ч2

In [ ]:
%%typecheck
# Повышение типа при переприсваивании
class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass

b = B()
reveal_type(b)
b = A()
reveal_type(b)
b.bm()

### Вывод типов при присваивании, ч3

In [ ]:
%%typecheck
# Понижение типа при переприсваивании

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass

a = A()
reveal_type(a)
a = B()
reveal_type(a)
#a.am()
a.bm()



### Вывод типов при присваивании, ч4

In [ ]:
%%typecheck
# Понижение типа при переприсваивании

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass

a = A()
reveal_type(a)
a = B()
reveal_type(a)
a = A()
reveal_type(a)
#a.am()
a.bm()


### Вывод типов при присваивании, ч5

In [ ]:
%%typecheck
# Задание сразу более общего типа

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass

a: A = B()
reveal_type(a)

a.bm()

a = A()
reveal_type(a)


### То же самое для стандартных типов

In [ ]:
%%typecheck
# Понижение типа при переприсваивании
a = 42.4
reveal_type(a)
a = 1
reveal_type(a)

a >> 10


In [ ]:
%%typecheck
# Повышение типа при переприсваивании
b = 1
reveal_type(b)
b = 42.4
reveal_type(b)

b >> 10

### Вызов функций, ч1

In [ ]:
%%typecheck
# Конкретный тип как параметр функции
# Вызов метода конкретного типа

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass


def f(value: B) -> None:
    value.bm()

f(B())
f(A())

### Вызов функций, ч2

In [ ]:
%%typecheck
# Конкретный тип как параметр функции
# Вызов метода общего типа

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass


def f(value: B) -> None:
    value.am()

f(B())
f(A())

### Вызов функций, ч3

In [ ]:
%%typecheck
# Общий тип как параметр функции
# Вызов метода конкретного типа

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass


def f(value: A) -> None:
    value.bm()

f(B())
f(A())

### Вызов функций, ч4

In [ ]:
%%typecheck
# Общий тип как параметр функции
# Вызов метода общего типа

class A:
    def am(self) -> None:
        pass

class B(A):
    def bm(self) -> None:
        pass


def f(value: A) -> None:
    reveal_type(value)
    value.am()

f(B())
f(A())

### Наследование, атрибуты класса, ч1

In [ ]:
%%typecheck
# Override c повышением типа

class A:
    VALUE = 4

class B(A):
    VALUE = 4.5
    

def f(p: A) -> None:
    print(p.VALUE >> 1)
    
f(A())
f(B())

### Наследование, атрибуты класса, ч2

In [ ]:
%%typecheck
# Override c понижением типа

class A:
    VALUE = 4.5

class B(A):
    VALUE = 4
    

def f(p: A) -> None:
    print(p.VALUE + 1)
    
f(A())
f(B())

### Наследование, атрибуты объекта

In [ ]:
%%typecheck
# Override c повышением типа

class A:
    def __init__(self) -> None:
        self.a = 1

class B(A):
    def __init__(self) -> None:
        self.a = 4.5

        
def f(p: A) -> None:
    print(p.a >> 5)
    
f(A())
f(B())

In [ ]:
%%typecheck
# Override c понижением типа

class A:
    def __init__(self) -> None:
        self.a = 4.5

class B(A):
    def __init__(self) -> None:
        self.a = 1

        
def f(p: A) -> None:
    print(p.a + 1)
    
f(A())
f(B())

### Наследование, переопределение методов, ч1

In [ ]:
%%typecheck
# Override c понижением типа параметра

class A:
    def am(self, a: float) -> int:
        return int(a)

class B(A):
    def am(self, a: int) -> int:
        return a >> 1

def f(p: A) -> None:
    p.am(1.1)

f(A())
f(B())

### Наследование, переопределение методов, ч2

In [ ]:
%%typecheck
# Override c повышением типа параметра

class A:
    def am(self, a: int) -> int:
        return int(a)

class B(A):
    def am(self, a: float) -> int:
        return int(a)


def f(p: A) -> None:
    p.am(1)

f(A())
f(B())

### Наследование, переопределение методов, возвращаемое значение, ч1

In [ ]:
%%typecheck
# Override c понижением типа возвращаемого значения

class A:
    def am(self, a: int) -> float:
        return float(a)

class B(A):
    def am(self, a: int) -> int:
        return a

def f(p: A) -> None:
    print(p.am(1))

f(A())
f(B())

### Наследование, переопределение методов, возвращаемое значение, ч2

In [ ]:
%%typecheck
# Override c понижением типа возвращаемого значения

class A:
    def am(self, a: int) -> int:
        return a

class B(A):
    def am(self, a: int) -> float:
        return float(a)

def f(p: A) -> None:
    b = p.am(1)
    b >> 1

f(A())
f(B())

# спасибо за внимание!

<center><img src="images/zeon.jpg" width=700px/></center>